In [ ]:
import os
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

PROMPT_TEMPLATES_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
PROMPT_TEMPLATES_DIR += '/config/api/prompt_templates/'

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
prompt_template = 'sythia_v1.3'
with open(PROMPT_TEMPLATES_DIR + f'{prompt_template}.txt', 'r') as f:
    prompt_template = f.read()

context = ''
question = 'How to fix a bike?'

prompt = prompt_template.format(context=context, question=question)
print(f'prompt len: {len(prompt)}\n')
print(prompt)

In [ ]:
model_id = 'migtissera/SynthIA-7B-v1.3'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    load_in_8bit=False,
    device_map='auto',
    resume_download=True,
)

pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    min_new_tokens=64,
    max_new_tokens=800,
    temperature=0.5,
    do_sample=True,
)

output_text = pipeline(prompt)[0]['generated_text']
output_text = output_text.replace(prompt+'\n', '')
print(output_text)